In [ ]:
from google.colab import files

uploaded = files.upload()



Saving emails.csv to emails.csv


In [ ]:
!pip install datasets


# Importing libraries, loading and transforming data

In [ ]:
import pandas as pd

df = pd.read_csv('emails.csv')

print(df.head())
df.size


                                                text  spam
0  Subject: naturally irresistible your corporate...     1
1  Subject: the stock trading gunslinger  fanny i...     1
2  Subject: unbelievable new homes made easy  im ...     1
3  Subject: 4 color printing special  request add...     1
4  Subject: do not have money , get software cds ...     1


11456

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, pipeline

df = pd.read_csv('emails.csv')

print(df.columns)

df = df.rename(columns={'spam': 'label'})  # Adjust if 'spam' is not the correct column

df['label'] = df['label'].map({0: 0, 1: 1})

dataset = Dataset.from_pandas(df)

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

print(tokenized_dataset.column_names)

model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

train_size = int(0.8 * len(tokenized_dataset))
train_dataset = tokenized_dataset.shuffle(seed=42).select([i for i in range(train_size)])
test_dataset = tokenized_dataset.shuffle(seed=42).select([i for i in range(train_size, len(tokenized_dataset))])

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

print("Training the model...")
trainer.train()

print("\nEvaluating the model...")
trainer.evaluate()

email_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

sample_emails = [
    "Congratulations! You've won a $1000 Walmart gift card. Click here to claim now.",
    "Meeting rescheduled to 3 PM tomorrow. Let me know if that works.",
    "Your Amazon order has been shipped and is expected to arrive by Monday.",
    "Exclusive offer: Get 50% off your next purchase! Use code SALE50 at checkout.",
    "Hey, just checking in to see how you're doing. Let me know if you'd like to catch up."
]

for i, email in enumerate(sample_emails, 1):
    result = email_pipeline(email)
    print(f"\nEmail {i}:")
    print(f"Text: {email}")
    print(f"Classification: {'Spam' if result[0]['label'] == 'LABEL_1' else 'Ham'}")
    print(f"Confidence: {result[0]['score']:.2f}")


Index(['text', 'spam'], dtype='object')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.110700,0.326642
2,0.045800,0.059130
3,0.003000,0.092156



Evaluating the model...


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



Email 1:
Text: Congratulations! You've won a $1000 Walmart gift card. Click here to claim now.
Classification: Spam
Confidence: 1.00

Email 2:
Text: Meeting rescheduled to 3 PM tomorrow. Let me know if that works.
Classification: Ham
Confidence: 1.00

Email 3:
Text: Your Amazon order has been shipped and is expected to arrive by Monday.
Classification: Spam
Confidence: 1.00

Email 4:
Text: Exclusive offer: Get 50% off your next purchase! Use code SALE50 at checkout.
Classification: Spam
Confidence: 1.00

Email 5:
Text: Hey, just checking in to see how you're doing. Let me know if you'd like to catch up.
Classification: Ham
Confidence: 1.00
